## Code for Creating SUMMA Forcing Files
* Taken from, at some point, Bart's code for creating SUMMA forcing files / ncdfs into the correct format 

In [1]:
# import packages 
# %matplotlib widget
%matplotlib inline

# plotting packages 
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns 

# interactive plotting
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots # adding for subplots
import plotly.figure_factory as ff

# data packages 
import pandas as pd
import numpy as np
import xarray as xr
from datetime import datetime

import csv 
import copy 
import os.path 

ModuleNotFoundError: No module named 'plotly'

In [4]:
ds =  pd.read_csv("/Users/Lumbr/OneDrive - UW/Documents/Washington/UnloadingRegimes/OtherSites/Sodankyla/SOD_1819_1hr_cleaned.csv")
# sod['datetime'] = pd.to_datetime(sod['datetime'])
# sod.index = pd.DatetimeIndex(sod['datetime'])
# sod.drop(columns=['datetime'], inplace=True)

ds.head()

,datetime,shortwave,longwave,ppt_1,ppt1,RH,windspeed,pressure,temp
0,2018-10-01 00:00:00,0.0,327.45,0.0,0.0,98.5,2.64420,97135.0,3.15
1,2018-10-01 01:00:00,0.0,321.75,0.0,0.0,97.5,2.23740,97185.0,2.45
2,2018-10-01 02:00:00,0.0,316.05,0.0,0.0,96.5,1.83060,97235.0,1.75
3,2018-10-01 03:00:00,0.0,312.60,0.0,0.0,96.0,1.72890,97295.0,1.10
4,2018-10-01 04:00:00,0.0,311.20,0.0,0.0,96.5,1.67805,97360.0,0.65


In [7]:
ds['precip'] = ds['ppt_1'].copy(deep=True)

In [ ]:
# Calculating the specific humidtiy from RH, temperature, and precip
def spechum(rh, T, p):
    T0 = 273.15
    return rh * np.exp((17.67*(T-T0))/(T-29.65)) / (0.263*p)

In [ ]:
sod_spechum = spechum(ds.RH, ds.temp_K, ds.air_press_Pa)

In [ ]:
shortwave = ds.shortwave
longwave = ds.longwave
precip = ds.precip
pressure = ds.air_press_Pa
airtemp = ds.temp_K
windspd = ds.windspeed
spechum = niwot_spechum
timestamp = ds.index

In [ ]:
# USING BARTS OLD CODE FOR CREATING 

# Attributes in the netCDF file, with their units and the full length name of the variable 
attrs = {
   'airpres':  {'units': 'Pa', 'long_name': 'Air pressure'},
   'airtemp':  {'units': 'K', 'long_name': 'Air temperature'},
   'spechum':  {'units': 'g g-1', 'long_name': 'Specific humidity'},
   'windspd':  {'units': 'Wind speed', 'long_name': 'm s-1'},
   'SWRadAtm': {'units': 'W m-2', 'long_name': 'Downward shortwave radiation'},
   'LWRadAtm': {'units': 'W m-2', 'long_name': 'Downward longwave radiation'},
   'pptrate':  {'units': 'kg m-2 s-1', 'long_name': 'Precipitation rate'}}

#mpl.rcParams['figure.figsize'] = (18, 10)

#______________________________________________________________________________________________

#bounds = pd.DatetimeIndex(pd.date_range('2016/10/01', '2017/05/17', freq='H'), name='time')[1:-1]
#ds['index'] = pd.DatetimeIndex(ds['timestamp'], name='time')
ds['index'] = pd.DatetimeIndex(timestamp, name='time')

# Need to change the latitude, longitude, and elevation
lats = [40.0329] 
lons = [-105.5464]
elev = 3050

bounds = ds.index

shape = (len(bounds), 1, )
dims = ('time', 'hru', )
coords = {'time': bounds}

# Time stepping
met_data = xr.Dataset(coords=coords)
met_data.time.encoding['calendar'] = 'standard'
met_data.time.encoding['units'] = 'hours since 1990-01-01'

# The variables in SUMMA that are in the forcings
summa_vars = ['airpres', 'airtemp', 'spechum', 
              'windspd', 'SWRadAtm', 'LWRadAtm', 'pptrate']
for varname in summa_vars:
    met_data[varname] = xr.DataArray(data=np.full(shape, np.nan),
                                     coords=coords, dims=dims,
                                     name=varname, attrs=attrs[varname])

In [ ]:
met_data['airpres' ].loc[{'hru': 0}] = pressure      #air pressure [Pa]
met_data['airtemp' ].loc[{'hru': 0}] = airtemp       #temperature [K]
met_data['windspd' ].loc[{'hru': 0}] = windspd       #wind speed [m s-1]
met_data['SWRadAtm'].loc[{'hru': 0}] = shortwave     #shortwave [W m-2]
met_data['LWRadAtm'].loc[{'hru': 0}] = longwave      #longwave [w m-2]
met_data['pptrate' ].loc[{'hru': 0}] = precip        #precip [kg m-2 s-1]
met_data['spechum' ].loc[{'hru': 0}] = spechum       #specific humidity [g/g-1]


# Using the attributes from the local_attributes netCDF instead of recreating them for the forcings 
#ds_local_attrs = xr.open_dataset('../data/local_attributes.nc')
#ds_template = xr.open_dataset('./CUES/cues_2016_summa_setup/forcings/cues2016.nc')

lsosmet = xr.open_dataset('/storage/lumbraca/SnowEx2017_metdata/Niwot_2016_fixed_final-Copy1.nc')
ds_local_attrs = lsosmet
met_data['hruId'] = ds_local_attrs['hruId']
met_data['latitude'] = ds_local_attrs['latitude']
met_data['longitude'] = ds_local_attrs['longitude']
met_data['data_step'] = [3600.]

met_data.to_netcdf('/storage/lumbraca/niwot_2017_ameriflux_data_FromBart.nc')

met_data